<a href="https://colab.research.google.com/github/milvus-io/bootcamp/blob/master/bootcamp/tutorials/quickstart/image_search_with_milvus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Search with Milvus

In this notebook, we will show you how to use Milvus to search for similar images in a dataset. We will use a subset of the [ImageNet](https://www.image-net.org/) dataset, then search for an image of an Afghan hound to demonstrate this.


## Dataset Preparation
First, we need to load the dataset and unextract it for further processing.

In [ ]:
!wget https://github.com/milvus-io/pymilvus-assets/releases/download/imagedata/reverse_image_search.zip
!unzip -q -o reverse_image_search.zip

## Prequisites

To run this notebook, you need to have the following dependencies installed:
- pymilvus>=2.4.2
- timm
- torch
- numpy
- sklearn
- pillow 

To run Colab, we provide the handy commands to install the necessary dependencies.

In [ ]:
!pip install grpcio==1.60
!pip install pymilvus==2.4.2
!pip install timm

## Define the Feature Extractor
Then, we need to define a feature extractor which extracts embedding from an image using timm's ResNet-34 model.

In [1]:

import torch
from PIL import Image
import timm
from sklearn.preprocessing import normalize
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
import numpy as np
from pymilvus import connections

class FeatureExtractor:
    def __init__(self, modelname):
        # Load the pre-trained model
        self.model = timm.create_model(modelname, pretrained=True, num_classes=0, global_pool='avg')
        self.model.eval()

        # Get the input size required by the model
        self.input_size = self.model.default_cfg['input_size']

        config = resolve_data_config({}, model=modelname)
        # Get the preprocessing function provided by TIMM for the model
        self.preprocess = create_transform(**config)

    def __call__(self, imagepath):
        # Preprocess the input image
        input_image = Image.open(imagepath).convert("RGB")  # Convert to RGB if needed
        input_image = self.preprocess(input_image)

        # Convert the image to a PyTorch tensor and add a batch dimension
        input_tensor = input_image.unsqueeze(0)

        # Perform inference
        with torch.no_grad():
            output = self.model(input_tensor)

        # Extract the feature vector
        feature_vector = output.squeeze().numpy()

        return normalize(feature_vector.reshape(1,-1), norm="l2").flatten()


## Create a Milvus Collection
Then we need to create Milvus collection to store the image embeddings

In [2]:
from pymilvus import MilvusClient, DataType

# Set up a Milvus client
client = MilvusClient(
    uri="example.db"
)
# Create a collection in quick setup mode
client.create_collection(
    collection_name="image_embeddings",
    vector_field_name="vector",
    dimension=512,
    auto_id=True,
    enable_dynamic_field=True,
    metric_type="COSINE"
)

## Insert the Embeddings to Milvus
We will extract embeddings of each image using the ResNet34 model and insert images from the training set into Milvus.

In [3]:
import os
extractor = FeatureExtractor('resnet34')

root = './train'
insert = True
if insert is True:
    for dirpath, foldername, filenames in os.walk(root):
        for filename in filenames:
            if filename.endswith('.JPEG'):
                filepath = dirpath+ '/' +filename
                image_embedding = extractor(filepath)
                client.insert("image_embeddings", {"vector": image_embedding, "filename": filepath})

## Search the Image
Now we can search the image using query embedding from a Afghan hound image in test set.

In [4]:
results = client.search("image_embeddings", data=[extractor('./test/Afghan_hound/n02088094_4261.JPEG')], output_fields=["filename"], search_params={"metric_type": "COSINE"})
for result in results:
    for hit in result:
        print(hit["distance"], hit["entity"]["filename"])

0.7624790072441101 ./train/Afghan_hound/n02088094_5911.JPEG
0.739625096321106 ./train/Afghan_hound/n02088094_6533.JPEG
0.7271758913993835 ./train/Afghan_hound/n02088094_2164.JPEG
0.7154390215873718 ./train/Bouvier_des_Flandres/n02106382_5429.JPEG
0.6918680667877197 ./train/Bouvier_des_Flandres/n02106382_6653.JPEG
0.680339515209198 ./train/Afghan_hound/n02088094_6565.JPEG
0.6768788695335388 ./train/Afghan_hound/n02088094_1045.JPEG
0.6709263920783997 ./train/Afghan_hound/n02088094_5532.JPEG
0.6666043996810913 ./train/Afghan_hound/n02088094_7360.JPEG
0.6618548631668091 ./train/soft-coated_wheaten_terrier/n02098105_400.JPEG


We can see that most of the images are from the same category as the search image, which is the Afghan hound. This means that we found similar images to the search image.